# Lab 3 - biblioteka pandas: analiza i wizualizacja danych



In [ ]:
import pandas as pd

In [ ]:
movie: pd.DataFrame = pd.read_csv('movie.csv')

movie.head()

## Analiza danych

### Rankingowanie obiektów według atrybutu

Operacja ta pozwala na wybór n wierszy, których wartość w wyznaczonej kolumnie jest największa bądź najmniejsza. W tym celu należy skorzystać z metod nlargest lub nsmallest wywoływanych na obiekcie klasy DataFrame, które zwrócą odpowiednio n wierszy o największej lub najmniejszej wartości według wskazanej kolumny. 

Przykład: wybór 10 filmów o największym budżecie:

In [ ]:
movie.nlargest(10, 'budget')[['movie_title', 'budget', 'imdb_score', 'genres', 'title_year']]

W analogiczny sposób można wyszukać 10 filmów o najmniejszym budżecie:

In [ ]:
movie.nsmallest(10, 'budget')[['movie_title', 'budget', 'imdb_score', 'genres', 'title_year']]

Łącząc wywołania obydwu metod można uzyskać podwójny efekt rankingowania obiektów - według dwóch atrybutów. Przykładowo, można wybrać spośród 10 najtańszych filmów, 5 z najwyższą oceną w serwisie IMDB:

In [ ]:
movie.nsmallest(10, 'budget')[['movie_title', 'budget', 'imdb_score', 'genres', 'title_year']].nlargest(5, 'imdb_score')

### Sortowanie ramek danych

Ramki danych (podobnie jak tabele w bazie danych) można sortować według jednej lub wielu kolumn. Służy do tego metoda sort_values wywoływana na obiekcie klasy DataFrame. 

Przykład: sortowanie malejące filmów według długości (kolumna duration).

In [ ]:
movie.sort_values('duration', ascending=False).head(15)

W podobny sposób można dokonać sortowania ramki według wielu kolumn. Należy jednak pamiętać o przekazaniu kolekcji nazw kolumn w metodzie sort_values. Poniższy przykład przedstawia sortowanie ramki według kolumny duration malejąco, a następnie według kolumny imdb_score:

In [ ]:
movie.sort_values(['duration', 'imdb_score'], ascending=False)[['movie_title', 'duration', 'imdb_score', 'genres']].head(20)

### Usuwanie duplikatów

Do usuwania duplikatów (wierszy o takich samych wartościach w wyznaczonej kolumnie) służy metoda drop_duplicates. Metoda (podobnie jak poprzednie) nie zastępuje oryginalnej ramki, lecz zwraca jej zmodyfikowaną kopię, zatem może posłużyć do przefiltrowania, lub pogrupowania oryginalnych danych. Przykładowo, znalezienie najwyżej ocenionego filmu w każdym roku:

In [ ]:
movie[['movie_title', 'imdb_score', 'title_year']].sort_values(['title_year', 'imdb_score'], ascending=False).drop_duplicates(subset='title_year')

### Agregacja danych

Agregacja danych polega na grupowaniu wierszy według wyznaczonej kolumny, a następnie na zastosowaniu funkcji agregacyjnej, która obliczy pewne wartości dla wyznaczonych atrybutów w pogrupowanych wierszach. Funkcją agregującą może być np. suma bądź średnia arytmetyczna. Do grupowania wierszy służy metoda groupby wywoływana na obiekcie klasy DataFrame oraz zwraca pogrupowaną ramkę w postaci obiektu klasy DataFrameGroupBy. Do zastosowania funkcji agregujących służy metoda agg wywoływana na obiekcie klasy DataFrameGroupBy.

Przykład: średnia ocena filmu w każdym roku.

In [ ]:
movie.groupby('title_year').agg({'imdb_score': 'mean'}).sort_values('title_year', ascending=False)

W podobny sposób można uzyskać łączny budżet filmów nakręconych przez każdego z reżyserów:

In [ ]:
movie.groupby('director_name').agg({'budget': 'sum'}).sort_values('budget', ascending=False)

Analogicznie można uzyskać dla każdego kraju liczbę wyprodukowanych filmów oraz średni i całkowity budżet:

In [ ]:
movie.groupby('country').agg({'budget': ['sum', 'mean'], 'movie_title': 'count'})

### Własne funkcje agregujące

Biblioteka pandas umożliwia wykorzystywanie własnych funkcji podczas agregacji danych. Taka funkcja powinna przyjmować jeden argument w postaci serii danych oraz zwracać dowolną wartośc skalarną.

Przykład: średni wynik filmu uzyskany w każdym roku przy użyciu własnej funkcji agregującej.

In [ ]:
def my_mean(x: pd.Series) -> float:
  return x.sum() / x.count()

In [ ]:
movie.groupby('title_year').agg({'imdb_score': my_mean}).sort_values('title_year', ascending=False)

### Filtrowanie danych

Do filtrowania danych służy metoda query wywoływana na obiekcie klasy DataFrame. Parametrem metody jest konstrukcja [nazwa_kolumny] [operator_logiczny] [wartosc], np. country == "Poland", która pozwala wybrać wszystkie wiersze pasujące do maski.

Przykład: wybór wszystkich filmów autorstwa Jamesa Camerona.

In [ ]:
movie.query('director_name == "James Cameron"')

Łącząc wyrażenia logiczne spójnikami and i or można konstruować złożone maski przeszukiwania danych w ramkach. 

Przykład: wszystkie filmy Jamesa Camerona wyprodukowane przed rokiem 1989.

In [ ]:
movie[['director_name', 'movie_title', 'title_year']].query('director_name == "James Cameron" and title_year < 1989')

## Wizualizacja danych

Biblioteka pandas udostępnia wygodny interfejs służący do wizualizacji danych znajdujących się w ramkach. Metody generujące wykresy w bibliotece pandas wykorzystują "pod spodem" bibliotekę matplotlib. W celu poprawnej prezentacji wykresu (niezależnie od środowiska) należy również zaimportować bibliotekę matplotlib. 



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

### Wykres liniowy

Najprostszą wersję wykresu liniowego można uzyskać wywołując metodę plot na serii danych. Wówczas na osi x wykresu znajdą się wartości pochodzące z indeksu wierszy prezentowanej ramki lub serii, a na osi y znajdą się docelowe wartości.

Przykład: wykres liniowy średniej długości filmu według roku

In [ ]:
ax: plt.Axes = movie.groupby('title_year').agg({'duration': 'mean'}).plot()
ax.set_ylabel('srednia dlugosc')

plt.show()

Biblioteka pandas umożliwia również wizualizację liniową ramek, gdzie wartości zostaną przedstawione w postaci kilku nakładających się wykresów liniowych.

Przykład: średnia długość oraz liczba filmów według roku.

In [ ]:
ax: plt.Axes = movie.groupby('title_year').agg({'duration': 'mean', 'movie_title': 'count'}).plot()

plt.show()

### Wykres kolumnowy

Do tworzenia wykresów kolumnowych w bibliotece pandas służy metoda bar.

Przykład: wykres kolumnowy średniej oceny filmu dla każdego języka

In [ ]:
ax: plt.Axes = movie.groupby('language').agg({'imdb_score': 'mean'}).plot.bar(figsize=(10, 5))
ax.set_ylabel('srednia ocena')

plt.show()

### Wykres kołowy

Do utworzenia wykresu kołowego służy metoda pie.

Przykład: wykres kołowy przedstawiający udział filmów kolorowych i czarno-białych pośród wszystkich filmów

In [ ]:
ax: plt.Axes = movie.groupby('color').agg({'color': 'count'}).plot.pie(subplots=True, autopct='%.2f %%', ylabel='', figsize=(5, 5))

plt.show()

## Zadania

1. Wczytać do ramki dane z pliku movie.csv.
2. Przygotować wykres słupkowy, w którym na osi x znajdzie się 10 lat, w których wystąpił najwyższy średni budżet filmów. Nadać tytuł osi y: "budget".
3. Przygotować wykres słupkowy przedstawiający oceny 20 najtańszych filmów. Na osi x umieścić tytuły filmów, a na osi y umieścić średnią ocenę.
4. Przygotować wykres liniowy przedstawiający sumę wartości w kolumnach: actor_1_facebook_likes, actor_2_facebook_likes, actor_3_facebook_likes w każdym roku.
5. Przygotować wykres liniowy przedstawiający zsumowane wartości w każdym roku dla kolumn: cast_total_facebook_likes oraz movie_facebook_likes dla wszystkich filmów, których budżet mieści się w przedziale [m-s, m+s], gdzie m oznacza średni budżet, a s oznacza odchylenie standardowe budżetu.
6. Przygotować wykres kołowy przedstawiający udział każdego reżysera w 20 najdroższych filmach.